In [32]:
import numpy as np
import time
from PIL import Image, ImageEnhance
import os
import sys
from numba import cuda
import matplotlib.pyplot as plt
from itertools import product

In [3]:
def generate_hooks(n_hooks, wheel_pixel_size):
    
    r = (wheel_pixel_size / 2) - 1
    
    theta = np.arange(n_hooks, dtype="float64") / n_hooks * (2 * np.pi)
    
    x = r * (1 + np.cos(theta)) + 0.5
    y = r * (1 + np.sin(theta)) + 0.5
    
    return np.array((x,y)).T

def generate_hooks_with_size(n_hooks, wheel_pixel_size, hook_pixel_size):
    
    r = (wheel_pixel_size / 2) - 1
    
    theta = np.arange(n_hooks, dtype="float64") / n_hooks * (2 * np.pi)
    
    epsilon = np.arcsin(hook_pixel_size / wheel_pixel_size)
    
    theta_acw = theta.copy() + epsilon
    theta_cw = theta.copy() - epsilon
    
    theta = np.stack((theta_cw, theta_acw)).ravel("F")
    
    x = r * (1 + np.cos(theta)) + 0.5
    y = r * (1 + np.sin(theta)) + 0.5
    
    return np.array((x,y)).T

def through_pixels(p0, p1):
    
    d = max(int(((p0[0]-p1[0])**2 + (p0[1]-p1[1])**2) ** 0.5), 1)
    
    pixels = p0 + (p1-p0) * np.array([np.arange(d+1), np.arange(d+1)]).T / d
    pixels = np.unique(np.round(pixels), axis=0).astype(int)
    
    return pixels


def build_through_pixels_dict(n_hooks):

    n_hook_sides = n_hooks

    l = [(0,1)]
    for j in range(n_hook_sides):
        for i in range(j):
            if j-i > 10 and j-i < (n_hook_sides - 10):
                l.append((i, j))    
                
    random_order = np.random.choice(len(l),len(l),replace=False)
    
    d = {}    
    t_list = []
    t0 = time.time()
    
    for n in range(len(l)):
        (i, j) = l[random_order[n]]
        p0, p1 = hooks[i], hooks[j]
        d[(i,j)] = through_pixels(p0, p1)
        
        t = time.time() - t0
        t_left = t * (len(l) - n - 1) / (n + 1)
        print(f"time left = {time.strftime('%M:%S', time.gmtime(t_left))}", end="\r")
    return d

In [4]:
n_hooks = 90
hook_real_size = 6 * (10**-3)
wheel_pixel_size = 3200
hooks = generate_hooks(n_hooks, wheel_pixel_size)
# hooks = generate_hooks_with_size(n_hooks, wheel_pixel_size, hook_real_size)
through_pixels_dict = build_through_pixels_dict(n_hooks)

In [5]:
test = np.zeros((3200, 3200))
print(test.shape)
count = 0
for pixels in through_pixels_dict.values():
    for pixel in pixels:
        count += 1
        test[pixel[0], pixel[1]] = 255
print(count)
image = Image.fromarray(test)
image.show()


(3200, 3200)
7351913


In [42]:
cwd = os.getcwd()
image = Image.open("ah_monochrome.jpg").convert("L").resize((wheel_pixel_size, wheel_pixel_size))
# Convert the image to a NumPy array
contrast = ImageEnhance.Contrast(image)
image = contrast.enhance(3)

image_array = np.array(image)



# Define the center and radius of the circle
cx, cy = 1600, 1600
r = 1600

# Create a grid of x and y coordinates
x, y = np.meshgrid(np.arange(image_array.shape[1]), np.arange(image_array.shape[0]))

# Compute the distance from each pixel to the center of the circle
distances = np.sqrt((x - cx)**2 + (y - cy)**2)

# Create a boolean mask that is True for pixels inside the circle
mask = distances < r

# Filter the pixel values using the mask
image_array[~mask] = 0

#scale up the image
# image_array = 255 - image_array.reshape((wheel_pixel_size, wheel_pixel_size))
image_array = image_array.reshape((wheel_pixel_size, wheel_pixel_size))

# Convert the NumPy array back to an image
new_image = Image.fromarray(image_array)

# Display the new image
new_image.show()




In [29]:
def compute_norm(): 
    results = {}
    for (i, j), pixels in through_pixels_dict.items():

        if i not in results:
             results[i] = (sys.maxsize, (0, 0))
        cur_min = results[i][0]
           
        # Compute test_line and ref using array indexing
        ref = image_array[pixels[:, 0], pixels[:, 1]]

        # Clip test_line between 0 and 255 and compute the norm
        arr_ref = np.array(ref)
        norm = np.sqrt(np.sum(arr_ref**2))
        
        # Update cur_min and (target_i, target_j) if necessary
        if norm < cur_min:
            results[i] = (norm, (i, j))

    return results

def compute_norm_best():
    progress = False
    was_better = 0
    was_not_better = 0
    for (i, j), pixels in through_pixels_dict.items():
        # Compute test_line and ref using array indexing
        ref = image_array[pixels[:, 0], pixels[:, 1]]
        pos = building[pixels[:, 0], pixels[:, 1]]

        # Clip test_line between 0 and 255 and compute the norm
        arr_ref = np.array(ref)
        arr_pos = np.array(pos)
        norm_with = np.linalg.norm(arr_ref)
        norm_without = np.linalg.norm(arr_pos - arr_ref)
        if norm_with < norm_without:
            # add line
            progress = True
            building[pixels[:, 0], pixels[:, 1]] = 0
            was_better += 1
        else:
            was_not_better += 1
    
    print(f"was better: {was_better}, was not better: {was_not_better}")
    return progress

def compute_norm_best2():
    cur_reduction = -1
    (target_i, target_j) = (-1, -1)
    for (i, j), pixels in through_pixels_dict.items():
        ref = image_array[pixels[:, 0], pixels[:, 1]]
        pos = building[pixels[:, 0], pixels[:, 1]]

        arr_ref = np.array(ref)
        arr_pos = np.array(pos)
        norm_with = np.linalg.norm(arr_ref)
        norm_without = np.linalg.norm(arr_pos - arr_ref)
        if cur_reduction < (norm_without - norm_with):
            cur_reduction = norm_without - norm_with
            (target_i, target_j) = (i, j)
            
    return (target_i, target_j)



In [8]:
# building = np.zeros((3200, 3200))
# building = np.full((3200, 3200), 255)
# through_pixels_dict = build_through_pixels_dict(n_hooks)

# max_iters = 30
# for num_iters in range(max_iters):
#     chosen_i = compute_norm()

#     for val in chosen_i.values():
#         #print("chosen_i val: ", val)
#         (i, j) = val[1]
#         pixels = through_pixels_dict[(i, j)]
        
#         # draw a black line
#         building[pixels[:, 0], pixels[:, 1]] = 0
#         through_pixels_dict.pop((i, j))

# image = Image.fromarray(building)
# image.show()

In [23]:
# n_hooks = 90
# hook_real_size = 6 * (10**-3)
# wheel_pixel_size = 3200
# hooks = generate_hooks(n_hooks, wheel_pixel_size)

# building = np.zeros((3200, 3200))
# building = np.full((3200, 3200), 255, dtype=np.uint8)

# image = Image.fromarray(building)
# #image.show()

# through_pixels_dict = build_through_pixels_dict(n_hooks)

# test = np.zeros((3200, 3200))
# count = 0
# for pixels in through_pixels_dict.values():
#     for pixel in pixels:
#         count += 1
#         test[pixel[0], pixel[1]] = 255
# print(count)
# image = Image.fromarray(test)
# #image.show()

# # Convert the NumPy array back to an image
# new_image = Image.fromarray(image_array)

# # Display the new image
# #new_image.show()

# max_iters = 1
# for i in range(max_iters):
#     # print(compute_norm_best())
#     if not compute_norm_best():
#         break
# print(building.shape)
# building.astype(np.uint8)
# image = Image.fromarray(building)
# image.show()

7351913ft = 00:00
was better: 2701, was not better: 405
(3200, 3200)


In [40]:
n_hooks = 90
hook_real_size = 6 * (10**-3)
wheel_pixel_size = 3200
hooks = generate_hooks(n_hooks, wheel_pixel_size)

building = np.zeros((3200, 3200))
building = np.full((3200, 3200), 255, dtype=np.uint8)

image = Image.fromarray(building)
#image.show()

through_pixels_dict = build_through_pixels_dict(n_hooks)

test = np.zeros((3200, 3200))
count = 0
for pixels in through_pixels_dict.values():
    for pixel in pixels:
        count += 1
        test[pixel[0], pixel[1]] = 255
print(count)
image = Image.fromarray(test)
#image.show()

# Convert the NumPy array back to an image
new_image = Image.fromarray(image_array)

# Display the new image
#new_image.show()

max_iters = 10000
for i in range(max_iters):
    (target_i, target_j) = compute_norm_best2()
    if (target_i, target_j) == (-1, -1):
        break
    # print((target_i, target_j))
    pixels = through_pixels_dict[(target_i, target_j)]
    # add line
    building[pixels[:, 0], pixels[:, 1]] = 0
    # remove line
    through_pixels_dict.pop((target_i, target_j))

print(building.shape)
building.astype(np.uint8)
image = Image.fromarray(building)
image.show()

7351913ft = 00:00
(3200, 3200)


## BELOW IS NOT MINE